In [2]:
import bamboolib as bam #easy to do data manipulation and highly recommend going to https://bamboolib.8080labs.com/ and that will help you follow along
import pandas as pd #data manipulation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

y=pd.read_csv(r'desktop/6040/Reviews.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'desktop/6040/Reviews.csv'

Rid Of Very Unhelpful Reviews & Narrow It To First 5000 Due To Data Size

In [59]:
train_set = y.loc[(y['Score'] > 2) & (y['Id'] < 15000)]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
...,...,...,...,...,...,...,...,...,...,...
14994,14995,B000EGZ99M,AMP2OWAY2BN0Z,Bonnie Suttle,1,1,4,1282694400,RiceSelect Orzo Whole Wheat Pasta,I've been using the product in dishes that cal...
14995,14996,B000EGZ99M,AH720G9X1MIQ8,KJT,1,1,5,1255824000,Rice Select Whole Wheat Orzo,Could no longer find Rice Select Whole Wheat O...
14996,14997,B000EGZ99M,A1MW2HEG4LF56B,S. Mariconda,3,4,5,1170633600,Absolutely wonderful comfort food!,I discovered this in our local QFC grocery sto...
14997,14998,B000EGZ99M,A1GY0FE07QFFLF,Arlington Cory,0,0,5,1326585600,Whole Wheat Orzo Tasty and Versatile,"Hard to find whole wheat orzo in supermarket, ..."


Bring in packages to get rid of Stopwords. This helps get rid of junk words that aren't meaningful

In [62]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JGrosz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JGrosz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Loop to strip out nouns

In [63]:
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

Now use the loop to strip out nouns from the Text column

In [64]:
data_nouns = pd.DataFrame(train_set.Text.apply(nouns))

Add Stop Words to remove words that aren't useful to analysis but common

In [71]:

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said','br']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


Turn Data Set Into A Matrix To Look Into Reviews (you will see in the last line we are only looking at the Text field)

In [72]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_nouns.Text)
data_dtmn = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtmn.index = data_nouns.index


In [73]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cv.vocabulary_.items())

Get ready to build model

In [74]:
from gensim import matutils, models
import scipy.sparse


In [75]:
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.049*"coffee" + 0.020*"flavor" + 0.018*"tea" + 0.017*"cup" + 0.016*"taste" + 0.014*"product" + 0.013*"chocolate" + 0.012*"amazon" + 0.010*"price" + 0.009*"cups"'),
 (1,
  '0.018*"treats" + 0.018*"dog" + 0.017*"food" + 0.013*"product" + 0.012*"dogs" + 0.009*"treat" + 0.008*"crackers" + 0.008*"bag" + 0.008*"taste" + 0.007*"amazon"')]